In [8]:
import pandas as pd
import numpy as np
from utils.hts_eda_utils import get_zero_columns, remove_zero_columns
from utils import hts_eda_utils

In [4]:
original_df = pd.read_excel('data/Quarterly_smoothing.xlsx', index_col=0)

original_df = remove_zero_columns(original_df)

Removing 30 columns with all zeros


## Let's Try MA, AR and ARIMA models separately, compare performance

In [5]:
df = original_df.copy()

# Set period type to Month Start (MS)
df.index = pd.date_range(start='2018-03-01', periods=len(df), freq='MS')  # Month start


In [6]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_arima_model(order):
    model = ARIMA(train, order=order)
    model_fit = model.fit()
    predictions = model_fit.predict(start=len(train), end=len(train) + len(test) - 1)
    mse = mean_squared_error(test, predictions)
    mae = mean_absolute_error(test, predictions)
    rmse = np.sqrt(mse)
    return mse, mae, rmse

In [12]:
def evaluate_series(series):
    train_size = int(len(series) * 0.7)
    train, test = series[:train_size], data[train_size:]


    print('Auto Regressive')
    mse_ar, mae_ar, rmse_ar = evaluate_arima_model((1, 0, 0))
    print('AR MSE:', mse_ar, '|', 'AR MAE:', mae_ar, '|', 'AR RMSE:', rmse_ar)

    print('Moving Average')
    mse_ma, mae_ma, rmse_ma = evaluate_arima_model((0, 0, 1))
    print('MSE:', mse_ma, '|', 'MAE:', mae_ma, '|', 'RMSE:', rmse_ma)

    print('Auto Regressive Integrated Moving Average')
    mse_arima, mae_arima, rmse_arima = evaluate_arima_model((1, 0, 1))
    print('ARIMA MSE:', mse_arima, '|', 'ARIMA MAE:', mae_arima, '|', 'ARIMA RMSE:', rmse_arima)

    return {
        'AR': {'MSE': mse_ar, 'MAE': mae_ar, 'RMSE': rmse_ar},
        'MA': {'MSE': mse_ma, 'MAE': mae_ma, 'RMSE': rmse_ma},
        'ARIMA': {'MSE': mse_arima, 'MAE': mae_arima, 'RMSE': rmse_arima}
    }


In [13]:
evaluate_series(df.iloc[:,0])

Auto Regressive
AR MSE: 1039291.0590480127 | AR MAE: 990.4492721364645 | AR RMSE: 1019.4562565642592
Moving Average
MSE: 1319916.8601176445 | MAE: 1144.0659262527527 | RMSE: 1148.8763467482672
Auto Regressive Integrated Moving Average
ARIMA MSE: 1155149.3979603036 | ARIMA MAE: 1051.3554520795733 | ARIMA RMSE: 1074.7787669843053


/Users/ariellubonja/Library/Python/3.9/lib/python/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


{'AR': {'MSE': 1039291.0590480127,
  'MAE': 990.4492721364645,
  'RMSE': 1019.4562565642592},
 'MA': {'MSE': 1319916.8601176445,
  'MAE': 1144.0659262527527,
  'RMSE': 1148.8763467482672},
 'ARIMA': {'MSE': 1155149.3979603036,
  'MAE': 1051.3554520795733,
  'RMSE': 1074.7787669843053}}